In [1]:
# Import the Modules into the Notebook
from uncertainties import ufloat
from uncertainties import wrap
from uncertainties.umath import * 
import numpy as np

In [2]:
# Setup the Structure

# Geometry
geometryDict = {}
geometryDict['beamHeight'] = ufloat(100, 0.25)
geometryDict['beamLength'] = ufloat(500, 0.2)
geometryDict['beamThick'] = ufloat(4.5, 0.1)
geometryDict['beamWidth'] = ufloat(50, 0.25)
print geometryDict

materialDict = {}
materialDict['elasticModulus'] = ufloat(200E9, 6.67E9)
materialDict['density'] = ufloat(8.8E-6, 0)
print materialDict

tubeDict = {}
tubeDict['tubePos'] = ufloat(0, 0)
tubeDict['tubeMass'] = ufloat(90, 0.08333)
tubeDict['gravity'] = ufloat(9.81, 0)
print tubeDict

beamDict = {}
beamDict['beamDeflection'] = ufloat(0, 0)
beamDict['beamFrequency'] = ufloat(0, 0)
beamDict['beamStress'] = ufloat(0, 0)
beamDict['beamMass'] = ufloat(0, 0)
print beamDict

sourceDict = {}
sourceDict['pos'] = ufloat(0, 0)
sourceDict['beamDistToCL'] = ufloat(600, 0.0667)
sourceDict['omega'] = ufloat(12.56, 0.041867)
sourceDict['force'] = ufloat(0, 0)
print sourceDict

detectorDict = {}
detectorDict['pos'] = ufloat(-0.5, 0.0333)
print detectorDict

xRayDict = {}
xRayDict['fsAlign'] = ufloat(0, 0)
print xRayDict

{'beamHeight': 100.0+/-0.25, 'beamWidth': 50.0+/-0.25, 'beamThick': 4.5+/-0.1, 'beamLength': 500.0+/-0.2}
{'elasticModulus': 200000000000.0+/-6670000000.0, 'density': 8.8e-06+/-0}
{'gravity': 9.81+/-0, 'tubeMass': 90.0+/-0.08333, 'tubePos': 0.0+/-0}
{'beamFrequency': 0.0+/-0, 'beamMass': 0.0+/-0, 'beamDeflection': 0.0+/-0, 'beamStress': 0.0+/-0}
{'force': 0.0+/-0, 'beamDistToCL': 600.0+/-0.0667, 'omega': 12.56+/-0.041867, 'pos': 0.0+/-0}
{'pos': -0.5+/-0.0333}
{'fsAlign': 0.0+/-0}


In [3]:
# To permit calling and transferring information across all subsystems
# Package it into an overall dict so that the
# Analytical and Response Surface can be programmed inside the functional blocks
overallSystemDict = {}
overallSystemDict['xRayDict'] = xRayDict
overallSystemDict['detectorDict'] = detectorDict
overallSystemDict['sourceDict'] = sourceDict
overallSystemDict['beamDict'] = beamDict
overallSystemDict['tubeDict'] = tubeDict
overallSystemDict['materialDict'] = materialDict
overallSystemDict['geometryDict'] = geometryDict
print overallSystemDict


{'detectorDict': {'pos': -0.5+/-0.0333}, 'materialDict': {'elasticModulus': 200000000000.0+/-6670000000.0, 'density': 8.8e-06+/-0}, 'sourceDict': {'force': 0.0+/-0, 'beamDistToCL': 600.0+/-0.0667, 'omega': 12.56+/-0.041867, 'pos': 0.0+/-0}, 'geometryDict': {'beamHeight': 100.0+/-0.25, 'beamWidth': 50.0+/-0.25, 'beamThick': 4.5+/-0.1, 'beamLength': 500.0+/-0.2}, 'tubeDict': {'gravity': 9.81+/-0, 'tubeMass': 90.0+/-0.08333, 'tubePos': 0.0+/-0}, 'xRayDict': {'fsAlign': 0.0+/-0}, 'beamDict': {'beamFrequency': 0.0+/-0, 'beamMass': 0.0+/-0, 'beamDeflection': 0.0+/-0, 'beamStress': 0.0+/-0}}


In [4]:
# Define Analytical Equations
def defineAnalyticalEquations_Lower(overallSystemDict):
    # Equation 1
    L = overallSystemDict['geometryDict']['beamLength']
    w = overallSystemDict['geometryDict']['beamWidth']
    t = overallSystemDict['geometryDict']['beamThick']
    h = overallSystemDict['geometryDict']['beamHeight']
    density = overallSystemDict['materialDict']['density']
    # User Enters Formula
    beamMass = density * L * (2 * (w + 2 * t)*t + 2 * h * t)
    
    # Equation 2
    tubeMass = overallSystemDict['tubeDict']['tubeMass']
    beamDistToCL = overallSystemDict['sourceDict']['beamDistToCL']
    gravity = overallSystemDict['tubeDict']['gravity']
    omega =  overallSystemDict['sourceDict']['omega']
    # User Enters Formula
    force = tubeMass * (0.001 * beamDistToCL * omega * omega + gravity)
    

    # Pushed back into the Dict
    overallSystemDict['beamDict']['beamMass'] = beamMass
    overallSystemDict['sourceDict']['force'] = force
    
    return overallSystemDict

In [5]:
# 'h', 'w', 't', 'L', 'E', 'F', 
# 'h^2', 'h w', 'h t', 'h L', 'h E', 'h F', 
# 'w^2', 'w t', 'w L', 'w E', 'w F', 
# 't^2', 't L', 't E', 't F', 
# 'L^2', 'L E', 'L F', 
# 'E^2', 'E F', 
# 'F^2'
def calculateResponseValue(h, w, t, L, E, F, coef, intercept):
    term1 = h*coef[0] + w*coef[1] + t*coef[2] + L*coef[3] + E*coef[4] + F*coef[5]
    term2 = h * (h*coef[6] + w*coef[7] + t*coef[8] + L*coef[9] + E*coef[10] + F*coef[11])
    term3 = w * (w*coef[12] + t*coef[13] + L*coef[14] + E*coef[15] + F*coef[16])

    term4 = t * (t*coef[17] + L*coef[18] + E*coef[19] + F*coef[20])
    term5 = L * (L*coef[21] + E*coef[22] + F*coef[23])
    term6 = E * (E*coef[24] + F*coef[25])
    term7 = F * F * coef[26]
    return intercept + term1 + term2 + term3 + term4 + term5 + term6 + term7
    

In [6]:
def calculateCodedValues(h, w, t, L, E, F):
    # 6 Variables: h, w, t, L, E, F
    lb = np.array([80, 40, 2, 400, 180E9, 9000])
    ub = np.array([120, 60, 7, 600, 220E9, 11000])
    avg = 0.5 * (lb + ub)

    loc_array = np.array([h, w, t, L, E, F])
    loc_array_coded = (loc_array - avg) / (ub - lb) * 2.0
    #print loc_array, loc_array_coded
    return list(loc_array_coded)
    
# Define Response Surface Equations
def defineResponseSurface(overallSystemDict, freq_rs, deflection_rs, stress_rs):
    L = overallSystemDict['geometryDict']['beamLength']
    w = overallSystemDict['geometryDict']['beamWidth']
    t = overallSystemDict['geometryDict']['beamThick']
    h = overallSystemDict['geometryDict']['beamHeight']
    E = overallSystemDict['materialDict']['elasticModulus']
    F = overallSystemDict['sourceDict']['force']
    
    h_c, w_c, t_c, L_c, E_c, F_c = calculateCodedValues(h, w, t, L, E, F)
    
    
    #x_vars = np.array([h, w, t, L, E, F])
    #x_vars_n = np.array([h.n, w.n, t.n, L.n, E.n, F.n])
    #x_vars_2d = x_vars.reshape(-1,1).transpose()
    #print x_vars_2d.shape
    # beamDeflection
    beamDeflection = calculateResponseValue(h, w, t, L, E, F, deflection_rs.steps[1][1].coef_[0,1:], deflection_rs.steps[1][1].intercept_[0])
    #beamDeflection = wrap(deflection_rs.predict)(x_vars_2d)
    #loc_I = (w * h * h * h/12)
    #beamDeflection = (1.0/(E * loc_I)) * F * L * L * L
    
    # beamFrequency
    beamFrequency = calculateResponseValue(h, w, t, L, E, F, freq_rs.steps[1][1].coef_[0,1:], freq_rs.steps[1][1].intercept_[0])
    #beamFrequency = wrap(freq_rs.predict)(x_vars_2d)
    #beamFrequency = E * w * h * h * h/12
    
    # beamStress
    beamStress = calculateResponseValue(h, w, t, L, E, F, stress_rs.steps[1][1].coef_[0,1:], stress_rs.steps[1][1].intercept_[0])
    #beamStress = wrap(stress_rs.predict)(x_vars_2d)
    #beamStress = F/(w * t)
    
    
    beamDict['beamDeflection'] = beamDeflection
    beamDict['beamFrequency'] = beamFrequency
    beamDict['beamStress'] = beamStress
    return overallSystemDict

In [7]:
# Define Analytical Equations
# Has some dependency on the Response Surface Equations.. So split it into two parts (Lower and Upper)
def defineAnalyticalEquations_Upper(overallSystemDict):
    # Equation 3
    tubePos = overallSystemDict['tubeDict']['tubePos']
    beamDeflection = overallSystemDict['beamDict']['beamDeflection']
    # User Enters Formula
    sourcePos = tubePos - beamDeflection
    
    # Equation 4
    detectorPos = overallSystemDict['detectorDict']['pos']
    # User Enters Formula
    fsAlign = detectorPos - sourcePos
    
    # Pushed back into the Dict
    overallSystemDict['sourceDict']['pos'] = sourcePos
    overallSystemDict['fsAlign'] = fsAlign
    return overallSystemDict
    

In [8]:
# Test out a simple calculation
from sklearn.externals import joblib

# Load the response surfaces
# https://stackoverflow.com/questions/34373606/scikit-learn-coefficients-polynomialfeatures
freq_rs = joblib.load('freq_rs.pkl')
print freq_rs.steps[1][1].coef_.shape
print freq_rs.steps[1][1].intercept_

deflection_rs = joblib.load('deflection_rs.pkl')
stress_rs = joblib.load('stress_rs.pkl')

overallSystemDict = defineAnalyticalEquations_Lower(overallSystemDict)
overallSystemDict = defineResponseSurface(overallSystemDict, freq_rs, deflection_rs, stress_rs)
overallSystemDict = defineAnalyticalEquations_Upper(overallSystemDict)
print overallSystemDict

(1, 28)
[27.50748961]
{'detectorDict': {'pos': -0.5+/-0.0333}, 'materialDict': {'elasticModulus': 200000000000.0+/-6670000000.0, 'density': 8.8e-06+/-0}, 'sourceDict': {'force': 9401.5944+/-57.46280148823399, 'beamDistToCL': 600.0+/-0.0667, 'omega': 12.56+/-0.041867, 'pos': -0.8701628007248919+/-0.02722088446317654}, 'geometryDict': {'beamHeight': 100.0+/-0.25, 'beamWidth': 50.0+/-0.25, 'beamThick': 4.5+/-0.1, 'beamLength': 500.0+/-0.2}, 'tubeDict': {'gravity': 9.81+/-0, 'tubeMass': 90.0+/-0.08333, 'tubePos': 0.0+/-0}, 'fsAlign': 0.37016280072489194+/-0.043010074993629174, 'xRayDict': {'fsAlign': 0.0+/-0}, 'beamDict': {'beamFrequency': 54.71261517436952+/-0.9967051363964022, 'beamMass': 6.2964+/-0.14852282230173783, 'beamDeflection': 0.8701628007248919+/-0.02722088446317654, 'beamStress': 112.07475227086393+/-2.0564129698467237}}


In [18]:
import scipy.stats as st

# Define DPMO and perform the optimization on variables
# Calculate the dpmo given the ufloat object and the lsl & usl
def dpo_lsl_usl(in_ufloat, lsl, usl):
    loc_mean = in_ufloat.nominal_value
    loc_std = in_ufloat.std_dev
    ret_dpo = 0.0

    if(~np.isnan(lsl)):
        z_lsl = (loc_mean - lsl) * 1.0/loc_std
        loc_lslcdf = st.norm.cdf(z_lsl)
        ret_dpo += 1 - loc_lslcdf
    
    if(~np.isnan(usl)):
        z_usl = (usl - loc_mean) * 1.0/loc_std
        loc_uslcdf = st.norm.cdf(z_usl)
        ret_dpo += 1.0 - loc_uslcdf
    
    # Find the Zoverall for the ret_dpo
    ret_zoverall = st.norm.ppf(1 - ret_dpo)

    return ret_dpo, ret_zoverall

In [19]:
import scipy.optimize as optimize

# params: h, w, L, t
def f(params, overallSystemDict=overallSystemDict):
    print(params)  # <-- you'll see that params is a NumPy array
    h_nom, w_nom, t_nom, L_nom = params # <-- for readability you may wish to assign names to the component variables
    # Set the values into the Overall System Dict
    L_std = (overallSystemDict['geometryDict']['beamLength']).std_dev
    w_std = (overallSystemDict['geometryDict']['beamWidth']).std_dev
    t_std = (overallSystemDict['geometryDict']['beamThick']).std_dev
    h_std = (overallSystemDict['geometryDict']['beamHeight']).std_dev
    
    overallSystemDict['geometryDict']['beamLength'] = ufloat(L_nom, L_std) 
    overallSystemDict['geometryDict']['beamWidth'] = ufloat(w_nom, w_std)
    overallSystemDict['geometryDict']['beamThick'] = ufloat(t_nom, t_std)
    overallSystemDict['geometryDict']['beamHeight'] = ufloat(h_nom, h_std)
    
    
    overallSystemDict = defineAnalyticalEquations_Lower(overallSystemDict)
    overallSystemDict = defineResponseSurface(overallSystemDict, freq_rs, deflection_rs, stress_rs)
    overallSystemDict = defineAnalyticalEquations_Upper(overallSystemDict)

    # Calculate the DPO of each variable
    beamFrequency = overallSystemDict['beamDict']['beamFrequency']
    beamStress = overallSystemDict['beamDict']['beamStress']
    beamMass = overallSystemDict['beamDict']['beamMass']
    fsAlign = overallSystemDict['fsAlign']
    
    # print beamFrequency, beamStress, beamMass, fsAlign
    
    dpo_beamFrequency, z_beamFrequency = dpo_lsl_usl(beamFrequency, np.nan, 55.0)
    #bounds_beamFrequency = (np.nan, 55)
    dpo_beamStress, z_beamStress = dpo_lsl_usl(beamStress, np.nan, 120)
    #bounds_beamStress = (np.nan, 20)
    dpo_beamMass, z_beamMass = dpo_lsl_usl(beamMass, np.nan, 8.0)
    #bounds_beamMass = (np.nan, 8)
    dpo_fsAlign, z_fsAlign = dpo_lsl_usl(fsAlign, -0.8, 0.8)
    #bounds_fsAlign = (0.4, 0.6)
    dpo_avg = 0.25 * (dpo_beamFrequency + dpo_beamStress + dpo_beamMass + dpo_fsAlign)

    print 'Frequency: %f, dpmo: %f, z: %f'%(beamFrequency.nominal_value, dpo_beamFrequency, z_beamFrequency)
    print 'Stress: %f, dpmo: %f, z: %f'%(beamStress.nominal_value, dpo_beamStress, z_beamStress)
    print 'fsAlign: %f, dpmo: %f, z: %f'%(fsAlign.nominal_value, dpo_fsAlign, z_fsAlign)
    print 'Mass: %f, dpmo: %f, z: %f'%(beamMass.nominal_value, dpo_beamMass, z_beamMass)
    
    
    return dpo_avg

h_Nom_init = 100 #(overallSystemDict['geometryDict']['beamHeight']).nominal_value
w_Nom_init = 50 #(overallSystemDict['geometryDict']['beamWidth']).nominal_value
t_Nom_init = 4.5 #(overallSystemDict['geometryDict']['beamThick']).nominal_value
L_Nom_init = 500 #(overallSystemDict['geometryDict']['beamLength']).nominal_value

initial_guess = [h_Nom_init, w_Nom_init, t_Nom_init, L_Nom_init]
bounds = np.c_[[80, 40, 2, 400], [120, 60, 7, 600]]
result = optimize.minimize(f, initial_guess, method='BFGS', bounds=bounds, options={'gtol': 1e-6, 'disp': True})
if result.success:
    fitted_params = result.x
    print(fitted_params)
else:
    raise ValueError(result.message)

[100.   50.    4.5 500. ]
Frequency: 54.712615, dpmo: 0.386545, z: 0.288335
Stress: 112.074752, dpmo: 0.000058, z: 3.853918
fsAlign: 0.370163, dpmo: 0.000000, z: inf
Mass: 6.296400, dpmo: 0.000000, z: inf
[100.00000001  50.           4.5        500.        ]
Frequency: 54.712615, dpmo: 0.386545, z: 0.288335
Stress: 112.074752, dpmo: 0.000058, z: 3.853918
fsAlign: 0.370163, dpmo: 0.000000, z: inf
Mass: 6.296400, dpmo: 0.000000, z: inf
[100.          50.00000001   4.5        500.        ]
Frequency: 54.712615, dpmo: 0.386545, z: 0.288335
Stress: 112.074752, dpmo: 0.000058, z: 3.853918
fsAlign: 0.370163, dpmo: 0.000000, z: inf
Mass: 6.296400, dpmo: 0.000000, z: inf
[100.          50.           4.50000001 500.        ]
Frequency: 54.712615, dpmo: 0.386545, z: 0.288335
Stress: 112.074752, dpmo: 0.000058, z: 3.853919
fsAlign: 0.370163, dpmo: 0.000000, z: inf
Mass: 6.296400, dpmo: 0.000000, z: inf
[100.          50.           4.5        500.00000001]
Frequency: 54.712615, dpmo: 0.386545, z: 0

Frequency: 53.093215, dpmo: 0.023017, z: 1.995076
Stress: 116.153476, dpmo: 0.035370, z: 1.807142
fsAlign: 0.407985, dpmo: 0.000000, z: inf
Mass: 5.966259, dpmo: 0.000000, z: inf
[ 99.54313756  50.02974954   4.2854047  500.20236245]
Frequency: 53.093215, dpmo: 0.023017, z: 1.995076
Stress: 116.153476, dpmo: 0.035370, z: 1.807142
fsAlign: 0.407985, dpmo: 0.000000, z: inf
Mass: 5.966259, dpmo: 0.000000, z: inf
[ 99.54313757  50.02974954   4.2854047  500.20236245]
Frequency: 53.093215, dpmo: 0.023017, z: 1.995076
Stress: 116.153476, dpmo: 0.035370, z: 1.807142
fsAlign: 0.407985, dpmo: 0.000000, z: inf
Mass: 5.966259, dpmo: 0.000000, z: inf
[ 99.54313756  50.02974956   4.2854047  500.20236245]
Frequency: 53.093215, dpmo: 0.023017, z: 1.995076
Stress: 116.153476, dpmo: 0.035370, z: 1.807142
fsAlign: 0.407985, dpmo: 0.000000, z: inf
Mass: 5.966259, dpmo: 0.000000, z: inf
[ 99.54313756  50.02974954   4.28540471 500.20236245]
Frequency: 53.093215, dpmo: 0.023017, z: 1.995076
Stress: 116.153476

Frequency: 49.367286, dpmo: 0.000000, z: 6.497166
Stress: 112.458512, dpmo: 0.000128, z: 3.655731
fsAlign: 0.486847, dpmo: 0.000000, z: 7.616924
Mass: 7.235250, dpmo: 0.000000, z: 5.434920
[ 81.99357801  51.72920345   5.58352386 508.15312302]
Frequency: 49.367286, dpmo: 0.000000, z: 6.497166
Stress: 112.458512, dpmo: 0.000128, z: 3.655731
fsAlign: 0.486847, dpmo: 0.000000, z: 7.616924
Mass: 7.235250, dpmo: 0.000000, z: 5.434920
[ 81.99357801  51.72920343   5.58352388 508.15312302]
Frequency: 49.367286, dpmo: 0.000000, z: 6.497165
Stress: 112.458511, dpmo: 0.000128, z: 3.655732
fsAlign: 0.486847, dpmo: 0.000000, z: 7.616924
Mass: 7.235250, dpmo: 0.000000, z: 5.434920
[ 81.99357801  51.72920343   5.58352386 508.15312303]
Frequency: 49.367286, dpmo: 0.000000, z: 6.497166
Stress: 112.458512, dpmo: 0.000128, z: 3.655731
fsAlign: 0.486847, dpmo: 0.000000, z: 7.616924
Mass: 7.235250, dpmo: 0.000000, z: 5.434920
[ 80.40271394  51.88339879   5.70360081 508.87390254]
Frequency: 49.044258, dpmo: 

Frequency: 48.860782, dpmo: 0.000000, z: 7.175650
Stress: 111.767179, dpmo: 0.000030, z: 4.013584
fsAlign: 0.497450, dpmo: 0.000000, z: 7.387068
Mass: 7.413102, dpmo: 0.000013, z: 4.200030
[ 79.36936992  51.98412894   5.78587336 509.34193808]
Frequency: 48.860782, dpmo: 0.000000, z: 7.175650
Stress: 111.767179, dpmo: 0.000030, z: 4.013585
fsAlign: 0.497450, dpmo: 0.000000, z: 7.387068
Mass: 7.413102, dpmo: 0.000013, z: 4.200030
[ 79.36936992  51.98412894   5.78587335 509.3419381 ]
Frequency: 48.860782, dpmo: 0.000000, z: 7.175650
Stress: 111.767179, dpmo: 0.000030, z: 4.013584
fsAlign: 0.497450, dpmo: 0.000000, z: 7.387068
Mass: 7.413102, dpmo: 0.000013, z: 4.200030
[ 79.43697494  51.97820555   5.78546928 509.31114761]
Frequency: 48.903455, dpmo: 0.000000, z: 7.117878
Stress: 111.703801, dpmo: 0.000026, z: 4.046576
fsAlign: 0.496480, dpmo: 0.000000, z: 7.408102
Mass: 7.415293, dpmo: 0.000014, z: 4.183010
[ 79.43697494  51.97820555   5.78546928 509.31114761]
Frequency: 48.903455, dpmo: 

Frequency: 50.648692, dpmo: 0.000001, z: 4.853459
Stress: 110.315863, dpmo: 0.000001, z: 4.776753
fsAlign: 0.457597, dpmo: 0.000000, z: 8.209536
Mass: 7.331904, dpmo: 0.000001, z: 4.705746
[ 83.72134153  51.58197079   5.60416999 507.36516243]
Frequency: 50.648693, dpmo: 0.000001, z: 4.853459
Stress: 110.315863, dpmo: 0.000001, z: 4.776753
fsAlign: 0.457597, dpmo: 0.000000, z: 8.209536
Mass: 7.331904, dpmo: 0.000001, z: 4.705746
[ 83.72134153  51.58197079   5.60416997 507.36516244]
Frequency: 50.648692, dpmo: 0.000001, z: 4.853459
Stress: 110.315863, dpmo: 0.000001, z: 4.776753
fsAlign: 0.457597, dpmo: 0.000000, z: 8.209536
Mass: 7.331904, dpmo: 0.000001, z: 4.705746
[ 83.81609856  51.57209514   5.59183602 507.32240612]
Frequency: 50.636012, dpmo: 0.000001, z: 4.869221
Stress: 110.425701, dpmo: 0.000001, z: 4.718438
fsAlign: 0.457942, dpmo: 0.000000, z: 8.209536
Mass: 7.318158, dpmo: 0.000001, z: 4.802077
[ 83.81609856  51.57209514   5.59183602 507.32240612]
Frequency: 50.636012, dpmo: 

In [ ]:
# Pretty Print the OverallDict

In [ ]:
# Variance Contribution